### Importing the necessary packages

In [2]:
# for data preprocessing
import numpy as np

# for data augmentation
from keras.preprocessing.image import ImageDataGenerator

# callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping

# for the convolutional neural network
from keras.models import Sequential
from keras.layers import Conv2D, Activation, Dropout, MaxPooling2D, Dense, Flatten

%tensorflow_version 1.14

ERROR! Session/line number was not unique in database. History logging moved to new session 60
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


Using TensorFlow backend.


### Loading the data
This notebook was run on the Google Colaboratory. I had uploaded the dataset as a npz file to my drive and accessed it from there. I used numpy for this. 

In [3]:
datasetpath = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/cats_vs_dogs'

def load_data(datasetpath):

    npzfile = np.load(datasetpath + '_training_data.npz')
    train_data = npzfile['arr_0']

    npzfile = np.load(datasetpath + '_training_labels.npz')
    train_labels = npzfile['arr_0']

    npzfile = np.load(datasetpath + '_test_data.npz')
    test_data = npzfile['arr_0']

    npzfile = np.load(datasetpath + '_test_labels.npz')
    test_labels = npzfile['arr_0']

    return ((train_data, train_labels), (test_data, test_labels))

##
((X_train, y_train), (X_test, y_test))  = load_data(datasetpath)
print('Training and Testing Data loaded.')

Training and Testing Data loaded.


### Preprocessing the data



In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

y_train = y_train.reshape(2000, 1)
y_test = y_test.reshape(1000, 1)
# X_train and X_test are in the correct shape already

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print('Data altered to match the required shape.')

(2000, 150, 150, 3)
(2000, 1)
(1000, 150, 150, 3)
(1000, 1)
Data altered to match the required shape.


### Defining hyperparameters for the model


In [5]:
batch_size = 16 
epochs = 25

train_size = X_train.shape[0]
test_size = X_test.shape[0]
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
channels = X_train.shape[3]

input_shape = (img_rows, img_cols, channels)

print(input_shape)

(150, 150, 3)


### Convolutional Neural Network

In [6]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=48, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
save_location = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/cats_vs_dogs.h5'

checkpoint = ModelCheckpoint(filepath=save_location, 
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

earlystopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)

callbacks = [checkpoint, earlystopping]

model.compile(optimizer='RMSprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

print(model.summary())



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 48)        13872     
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 48)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 48)        0         
________________________

In [9]:
history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

ERROR! Session/line number was not unique in database. History logging moved to new session 61
Train on 2000 samples, validate on 1000 samples
Epoch 1/25
2000/2000 [==============================] - 71s 36ms/step - loss: 0.7169 - acc: 0.5350 - val_loss: 0.6747 - val_acc: 0.6350

Epoch 00001: val_loss improved from inf to 0.67475, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/cats_vs_dogs.h5
Epoch 2/25
2000/2000 [==============================] - 71s 35ms/step - loss: 0.6729 - acc: 0.6300 - val_loss: 0.6279 - val_acc: 0.5910

Epoch 00002: val_loss improved from 0.67475 to 0.62789, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/cats_vs_dogs.h5
Epoch 3/25
2000/2000 [==============================] - 71s 36ms/step - loss: 0.6050 - acc: 0.6930 - val_loss: 0.5677 - val_acc: 0.7090

Epoch 00003: val_loss improved from 0.62789 to 0.56773, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/c

### Boosting Accuracy with Data Augmentation

In [10]:
train_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/datasets/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/1. Cats vs Dogs/datasets/test'

num_of_train_samples = 2000
num_of_test_samples = 1000

train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 30,
    width_shift_range = 0.3,
    height_shift_range =  0.3,
    horizontal_flip = True,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(
    rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True)

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir, 
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False)

Found 2005 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = num_of_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data=validation_generator,
    validation_steps = num_of_test_samples // batch_size)

# // is the floor division - returns integer

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/25
125/125 [==============================] - 1191s 10s/step - loss: 0.6617 - acc: 0.6107 - val_loss: 0.6480 - val_acc: 0.6250

Epoch 00001: val_loss did not improve from 0.56773
Epoch 2/25
125/125 [==============================] - 94s 754ms/step - loss: 0.6569 - acc: 0.6244 - val_loss: 0.6490 - val_acc: 0.6179

Epoch 00002: val_loss did not improve from 0.56773
Epoch 3/25
125/125 [==============================] - 94s 753ms/step - loss: 0.7096 - acc: 0.6295 - val_loss: 0.6366 - val_acc: 0.6443

Epoch 00003: val_loss did not improve from 0.56773
Epoch 4/25
125/125 [==============================] - 93s 746ms/step - loss: 0.6600 - acc: 0.6078 - val_loss: 0.6319 - val_acc: 0.6453

Epoch 00004: val_loss did not improve from 0.56773
Epoch 5/25
125/125 [==============================] - 93s 744ms/step - loss: 0.6468 - acc: 0.6504 - val_loss: 0.6474 - val_acc: 0.6199

Epoch 00005: val_loss did not improve from 0.56773
Epoch 6/25
125/125 [==============================] - 93s 746ms/s